## KELOMPOK 1

# Tugas Kelompok

Download data disini :

https://drive.google.com/file/d/1yRYkjsF9CuC4RXueZQCHuJnXRSXliBjS/view?usp=sharing

## Loading Dataset

In [ ]:
import pandas as pd

# Path ke file Excel
data = '/content/911_Calls_for_Service_(Last_30_Days).csv'

# Baca file CSV (bukan Excel) menggunakan pd.read_csv
dataku = pd.read_csv(data)

# Tampilkan lima baris pertama data
print(dataku.head())


           X          Y   incident_id agency               incident_address  \
0 -82.993989  42.410293  202306901627   DPD     Glenfield Ave & Gratiot Ave   
1 -83.052792  42.415266  202306901629   DPD            Luce St & Fenelon St   
2 -83.072103  42.368421  202306901630   DPD   W Baltimore St & Woodward Ave   
3 -83.226541  42.429892  202306901632   DPD          Avon Ave & W 7 Mile Rd   
4 -83.240073  42.409465  202306901633   DPD     Heyden St & Grand River Ave   

   zip_code priority callcode             calldescription  category  ...  \
0     48213        3   W4807   START OF SHIFT INFORMATION  STRTSHFT  ...   
1     48212        3   SA               SPECIAL ATTENTION  SPCL ATT  ...   
2     48202        3   SA               SPECIAL ATTENTION  SPCL ATT  ...   
3     48219        4   372040                 UDAA REPORT  UDAAREPT  ...   
4     48219        3   SA               SPECIAL ATTENTION  SPCL ATT  ...   

  traveltime totalresponsetime time_on_scene totaltime     neighborh

Bagian 1: Model Populasi Polisi Detroit

Baca data dari file Laporan Polisi Detroit menggunakan modul csv dan terjemahkan data ke dalam daftar dictionary. Menggunakan Filter dengan fungsi lambda untuk mengecualikan dictionary (baris CSV) yang memiliki data yang hilang di kolom Zip, atau kolom Neighborhood. Dengan menggunakan fungsi lambda dan Reduce, hitung total waktu respons rata-rata, waktu pengiriman rata-rata, dan total waktu rata-rata untuk kepolisian Detroit.

In [ ]:
import csv

# Fungsi untuk membaca file CSV dan memfilter baris yang lengkap
def baca_csv_filter(nama_file):
    with open(nama_file, 'r') as file:
        pembaca = csv.DictReader(file)
        data = filter(lambda x: x['zip_code'] != '' and x['neighborhood'] != '', pembaca)
        return list(data)

# Fungsi untuk mengubah nilai menjadi float jika memungkinkan, atau mengembalikan 0
def aman_ke_float(nilai):
    try:
        return float(nilai)
    except (ValueError, TypeError):
        return 0.0

# Menghitung total waktu respons rata-rata, waktu pengiriman rata-rata, dan total waktu rata-rata
def hitung_rerata(data):
    total_waktu_respons = 0
    total_waktu_pengiriman = 0
    total_waktu = 0
    jumlah_rekaman = 0

    for baris in data:
        total_waktu_respons += aman_ke_float(baris.get('totalresponsetime'))
        total_waktu_pengiriman += aman_ke_float(baris.get('traveltime'))
        total_waktu += aman_ke_float(baris.get('totaltime'))
        jumlah_rekaman += 1

    if jumlah_rekaman == 0:
        return {
            'neighborhood': 'DETROIT',  # Tambahkan kunci neighborhood dengan nilai 'DETROIT'
            'rerata_waktu_respons': 0,
            'rerata_waktu_pengiriman': 0,
            'rerata_total_waktu': 0
        }

    return {
        'neighborhood': 'DETROIT',  # Tambahkan kunci neighborhood dengan nilai 'DETROIT'
        'rerata_waktu_respons': total_waktu_respons / jumlah_rekaman,
        'rerata_waktu_pengiriman': total_waktu_pengiriman / jumlah_rekaman,
        'rerata_total_waktu': total_waktu / jumlah_rekaman
    }

# Path file CSV
path_csv = "/content/911_Calls_for_Service_(Last_30_Days).csv"

# Baca file CSV dan filter data
data_terfilter = baca_csv_filter(path_csv)

# Hitung rata-rata waktu respons, waktu pengiriman, dan total waktu
hasil_part1 = hitung_rerata(data_terfilter)
print(hasil_part1)


{'neighborhood': 'DETROIT', 'rerata_waktu_respons': 8.023254124066208, 'rerata_waktu_pengiriman': 2.2458019938298404, 'rerata_total_waktu': 28.634880548686127}


Bagian 2: Modelkan Neighborhood Samples

Menggunakan fungsi lambda dan map, atau lambda dan Filter, bagi list dictionary menjadi list dictionary yang lebih kecil yang dipisahkan oleh neighborhood. Dengan menggunakan lambda dan Reduce, temukan total waktu respons rata-rata untuk setiap neighborhood, waktu pengiriman rata-rata untuk setiap neighborhood, dan total waktu rata-rata untuk setiap neighborhood dan simpanlah ke dalam list dictionary. Tambahkan item dictioanary untuk menyertakan data populasi untuk semua Detroit dalam daftar gabungan Anda.

In [6]:
import csv
from functools import reduce

# Fungsi untuk membaca file CSV dan memfilter baris yang lengkap
def baca_csv_saring(nama_file):
    with open(nama_file, 'r') as file:
        pembaca = csv.DictReader(file)
        data = filter(lambda x: x['zip_code'] != '' and x['neighborhood'] != '', pembaca)
        return list(data)

# Fungsi untuk mengubah nilai menjadi float jika memungkinkan, atau mengembalikan 0
def aman_ke_float(nilai):
    try:
        return float(nilai)
    except (ValueError, TypeError):
        return 0.0

# Mengelompokkan data berdasarkan daerah (neighborhood)
def kelompokkan_berdasarkan_daerah(data):
    return reduce(lambda acc, baris: acc.setdefault(baris['neighborhood'], []).append(baris) or acc, data, {})

# Menghitung rata-rata waktu respons, waktu perjalanan, dan total waktu untuk setiap daerah
def hitung_rata_rata_daerah(data):
    rata_rata = {}
    for daerah, catatan in data.items():
        if not catatan:
            rata_rata[daerah] = {
                'rata_rata_waktu_respons': 0,
                'rata_rata_waktu_perjalanan': 0,
                'rata_rata_waktu_total': 0,
                'populasi': 0
            }
        else:
            jumlah_catatan = len(catatan)
            total_waktu_respons = sum(aman_ke_float(baris['totalresponsetime']) for baris in catatan)
            total_waktu_perjalanan = sum(aman_ke_float(baris['traveltime']) for baris in catatan)
            total_waktu = sum(aman_ke_float(baris['totaltime']) for baris in catatan)

            rata_rata[daerah] = {
                'rata_rata_waktu_respons': total_waktu_respons / jumlah_catatan,
                'rata_rata_waktu_perjalanan': total_waktu_perjalanan / jumlah_catatan,
                'rata_rata_waktu_total': total_waktu / jumlah_catatan,
                'populasi': jumlah_catatan  # Menambahkan populasi setiap daerah
            }
    return rata_rata

# Path file CSV
path_csv = "/content/911_Calls_for_Service_(Last_30_Days).csv"

# Baca file CSV dan filter data
data_terfilter = baca_csv_saring(path_csv)

# Bagian 2 - Kelompokkan data berdasarkan daerah
data_terkelompok = kelompokkan_berdasarkan_daerah(data_terfilter)

# Bagian 3 - Hitung rata-rata waktu respons, waktu perjalanan, dan total waktu untuk setiap daerah
rata_rata_daerah = hitung_rata_rata_daerah(data_terkelompok)

# Tambahkan item dictionary untuk menyertakan data populasi untuk semua Detroit dalam daftar gabungan
total_populasi = len(data_terfilter)
for daerah in rata_rata_daerah:
    rata_rata_daerah[daerah]['total_populasi'] = total_populasi

print(rata_rata_daerah)


{'Eden Gardens': {'rata_rata_waktu_respons': 13.271794871794876, 'rata_rata_waktu_perjalanan': 2.132478632478633, 'rata_rata_waktu_total': 39.54829059829062, 'populasi': 234, 'total_populasi': 78442}, 'Buffalo Charles': {'rata_rata_waktu_respons': 15.238418079096045, 'rata_rata_waktu_perjalanan': 3.449717514124294, 'rata_rata_waktu_total': 43.76610169491524, 'populasi': 177, 'total_populasi': 78442}, 'New Center': {'rata_rata_waktu_respons': 1.5535570469798656, 'rata_rata_waktu_perjalanan': 0.490738255033557, 'rata_rata_waktu_total': 15.192617449664418, 'populasi': 745, 'total_populasi': 78442}, "O'Hair Park": {'rata_rata_waktu_respons': 14.337538461538458, 'rata_rata_waktu_perjalanan': 4.272307692307694, 'rata_rata_waktu_total': 44.951076923076954, 'populasi': 325, 'total_populasi': 78442}, 'Miller Grove': {'rata_rata_waktu_respons': 11.465137614678897, 'rata_rata_waktu_perjalanan': 2.105963302752293, 'rata_rata_waktu_total': 32.55366972477064, 'populasi': 218, 'total_populasi': 78442

Bagian 3: Buat file Output JSON

Menggunakan modul JSON, format list dictionary Anda sebagai JSON

In [ ]:
import json

# Bagian 3: Buat file Output JSON
def simpan_file_ke_json(data_part1, data_daerah, nama_file):
    with open(nama_file, 'w') as berkas_json:
        json.dump({'hasil_part1': data_part1, 'rata_rata_daerah': data_daerah}, berkas_json, indent=4)

# Path untuk menyimpan file JSON
path_output_json = "output.json"

# Simpan data ke dalam file JSON
simpan_file_ke_json(hasil_part1, rata_rata_daerah, path_output_json)

print("Data telah disimpan dalam file JSON:", path_output_json)


Data telah disimpan dalam file JSON: output.json
